In [85]:
#Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Scikit Learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
from torchvision import datasets
from torchvision.transforms import ToTensor, transforms
from torch.utils.data import Dataset, DataLoader

from PIL import Image

In [86]:
# Load data
trainDataNP = np.load("fashion_train.npy")
testDataNP = np.load("fashion_test.npy")

# Split data into X and y arrays
X_train = trainDataNP[:, :-1]
y_train = trainDataNP[:, -1]
X_test = testDataNP[:, :-1]
y_test = testDataNP[:, -1]

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# CNN

In [134]:
# Define variables
batch_size = 64
n_classes = len(np.unique(y_train))
alpha = 0.001 # Learning rate
n_epochs = 100

# Create device object to choose whether to run training on CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [135]:
# Define superclass to handle own input datasets
class CustomDataset(Dataset):
    def __init__(self, X, y, transform = None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):

        # Convert NumPy array to PIL Image
        image = Image.fromarray(self.X[idx])

        # Collect image and label in sample dict
        sample = {'image': image, 'label': self.y[idx]}

        # Use transformer
        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [136]:
# Define transformer
transformer = transforms.Compose([transforms.Resize((28, 28)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5], std=[0.5])
])

In [137]:
# Create dataset
train_data = CustomDataset(X_train_scaled, y_train, transform = transformer)

# Create data loader
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)

In [138]:
# Create custom CNN model
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()

        # First convolutional layer: 1 input channel (greyscale), 32 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 3)

        # Second convolutional layer: 32 input channel, 32 output channels, 3x3 kernel
        self.conv2 = nn.Conv2d(32, 32, kernel_size = 3)

        # First pooling layer
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        # Third convolutional layer: 32, input channels, 64 output channels, 3x3 kernel
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)

        # Third convolutional layer: 64, input channels, 64 output channels, 3x3 kernel
        self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)

        # Second pooling layer
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        # Fully connected layers
        # Flatten before passing to fully connected layers
        self.fc_input_size = self.calculate_fc_input_size()
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, n_classes)

    def calculate_fc_input_size(self):
        # Dummy input to calculate the size after convolutional layers
        x = torch.randn(1, 1, 28, 28)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        # Flatten before passing to fully connected layers
        return x.view(1, -1).size(1)

    def forward(self, x):

        # First convolutional block
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        
        # Second convolutional block
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        
        # Flatten before passing to fully connected layers
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)

        return x

In [139]:
# Instantiate model
model = CNN(n_classes = n_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = alpha, weight_decay = 0.005, momentum = 0.9)  

# Defnie total_step to ease iteration through batches
total_step = len(train_loader)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [140]:
# Train model

# Use defined number of epochs to determine amount of iterations to train model on 
for epoch in range(1, n_epochs + 1):

    # Load in the data in batches using dataloader object
    for batch in train_loader:

        # Extract images and labels from the batch
        images, labels = batch['image'], batch['label']

        # Pass image and label tensors to device
        images = images.to(device)
        labels = labels.to(device)

        # Run forward pass
        output = model(images)
        loss = criterion(output, labels)

        # Run backwards pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print results
    print(f'Epoch {epoch} / {n_epochs}, Loss: {loss.item()}')

Epoch 1 / 20, Loss: 1.2007768154144287
Epoch 2 / 20, Loss: 1.1654188632965088
Epoch 3 / 20, Loss: 1.4722650051116943
Epoch 4 / 20, Loss: 1.3363670110702515
Epoch 5 / 20, Loss: 0.9191613793373108
Epoch 6 / 20, Loss: 0.7635058760643005
Epoch 7 / 20, Loss: 1.2439160346984863
Epoch 8 / 20, Loss: 1.0293141603469849
Epoch 9 / 20, Loss: 1.16072416305542
Epoch 10 / 20, Loss: 0.737480640411377
Epoch 11 / 20, Loss: 1.1014708280563354
Epoch 12 / 20, Loss: 0.5376147031784058
Epoch 13 / 20, Loss: 0.6332538723945618
Epoch 14 / 20, Loss: 1.2123373746871948
Epoch 15 / 20, Loss: 0.8293539881706238
Epoch 16 / 20, Loss: 0.8185349702835083
Epoch 17 / 20, Loss: 0.6440213918685913
Epoch 18 / 20, Loss: 0.5289050340652466
Epoch 19 / 20, Loss: 0.790256679058075
Epoch 20 / 20, Loss: 0.551320493221283


In [141]:
# from lda import LDA
# MDA_4 = LDA(n_components = 4)

# X_projected_4 = MDA_4.fit_transform(X_train_scaled, y_train)
# X_t_projected_4 = MDA_4.transform(X_test_scaled)

In [142]:
# cnn = MLPClassifier()
# cnn.fit(X_projected_4, y_train)

In [143]:
# cnn.score(X_projected_4, y_train)

In [144]:
# y_pred = cnn.predict(X_t_projected_4)

# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

In [145]:
# for n in range (1,10):
#     cnn = MLPClassifier()
#     scores = cross_val_score(cnn, X_train, y_train, cv=5)
#     print(n, round(np.mean(scores), 4), 'and', accuracy)